In [1]:
import pandas as pd

In [2]:
feature_dir = 'data/features/'

In [3]:
df_twitter = pd.read_csv(feature_dir+'twitter_features.csv')
df_news = pd.read_csv(feature_dir+'news_features.csv')
df_out= pd.read_csv(feature_dir+'stock_features.csv')

In [4]:
df = pd.merge(df_out, df_twitter, how='left', on='Date')
df2 = pd.merge(df_out, df_news, how='left', on='Date')


In [5]:
df.corr()

,Change VIX,Lag 2 Change VIX,Lag 2 Significant VIX,Change SPY,Lag 2 Change SPY,Lag 2 Significant SPY,BarackObama_sentiment_score,BarackObama_heuristic_score,cnnbrk_sentiment_score,cnnbrk_heuristic_score,KimKardashian_sentiment_score,KimKardashian_heuristic_score
Change VIX,1.000000,-0.044397,-0.096219,-0.758898,0.012864,-0.013647,0.021521,0.029132,0.043728,0.038093,0.047395,0.037807
Lag 2 Change VIX,-0.044397,1.000000,0.746575,0.054019,-0.812895,-0.695892,-0.018619,-0.024138,0.169578,0.161313,-0.080551,-0.091466
Lag 2 Significant VIX,-0.096219,0.746575,1.000000,0.078564,-0.604858,-0.526078,0.043486,0.041407,0.175315,0.180703,-0.050814,-0.066612
Change SPY,-0.758898,0.054019,0.078564,1.000000,-0.097094,-0.064839,-0.045561,-0.047821,0.083666,0.096154,-0.045535,-0.035323
Lag 2 Change SPY,0.012864,-0.812895,-0.604858,-0.097094,1.000000,0.822793,0.012597,0.015496,-0.156854,-0.142991,0.019596,0.030105
Lag 2 Significant SPY,-0.013647,-0.695892,-0.526078,-0.064839,0.822793,1.000000,0.073112,0.070065,-0.151014,-0.134161,-0.008958,-0.003135
BarackObama_sentiment_score,0.021521,-0.018619,0.043486,-0.045561,0.012597,0.073112,1.000000,0.995785,-0.006240,0.014262,0.069529,0.069534
BarackObama_heuristic_score,0.029132,-0.024138,0.041407,-0.047821,0.015496,0.070065,0.995785,1.000000,-0.019025,0.000987,0.080592,0.081671
cnnbrk_sentiment_score,0.043728,0.169578,0.175315,0.083666,-0.156854,-0.151014,-0.006240,-0.019025,1.000000,0.994058,0.198882,0.196652
cnnbrk_heuristic_score,0.038093,0.161313,0.180703,0.096154,-0.142991,-0.134161,0.014262,0.000987,0.994058,1.000000,0.227692,0.225898


In [6]:
df2.corr()

,Change VIX,Lag 2 Change VIX,Lag 2 Significant VIX,Change SPY,Lag 2 Change SPY,Lag 2 Significant SPY,New York Times_sentiment_score,CNN_sentiment_score,Washington Post_sentiment_score
Change VIX,1.000000,-0.044397,-0.096219,-0.758898,0.012864,-0.013647,-0.065055,0.066248,-0.062777
Lag 2 Change VIX,-0.044397,1.000000,0.746575,0.054019,-0.812895,-0.695892,-0.107881,0.080929,-0.036708
Lag 2 Significant VIX,-0.096219,0.746575,1.000000,0.078564,-0.604858,-0.526078,-0.064139,0.089364,-0.013954
Change SPY,-0.758898,0.054019,0.078564,1.000000,-0.097094,-0.064839,0.044865,-0.058979,0.031722
Lag 2 Change SPY,0.012864,-0.812895,-0.604858,-0.097094,1.000000,0.822793,0.117475,-0.082842,0.046491
Lag 2 Significant SPY,-0.013647,-0.695892,-0.526078,-0.064839,0.822793,1.000000,0.083786,-0.092123,0.056981
New York Times_sentiment_score,-0.065055,-0.107881,-0.064139,0.044865,0.117475,0.083786,1.000000,-0.042060,0.021621
CNN_sentiment_score,0.066248,0.080929,0.089364,-0.058979,-0.082842,-0.092123,-0.042060,1.000000,0.224250
Washington Post_sentiment_score,-0.062777,-0.036708,-0.013954,0.031722,0.046491,0.056981,0.021621,0.224250,1.000000


In [7]:
#OLD

In [8]:
data_VIX = pd.read_csv('VIX.csv', parse_dates=['Date'])
data_Obama = pd.read_csv('BarackObama.csv', parse_dates=['Date'])
data_NYT = pd.read_csv('New York Times.csv', parse_dates=['Date'])

FileNotFoundError: [Errno 2] File b'VIX.csv' does not exist: b'VIX.csv'

In [ ]:
data_Obama['Date']= pd.to_datetime(data_Obama['Date'].dt.normalize())
data_VIX['Date']= pd.to_datetime(data_VIX['Date'].dt.normalize())
data_NYT['Date']= pd.to_datetime(data_NYT['Date']).dt.normalize()

In [ ]:
data_NYT

In [ ]:
data_Obama

In [ ]:
data_Obama = data_Obama[data_Obama['Date']<last_date]
data_Obama = data_Obama[data_Obama['Date']>first_date]
data_VIX = data_VIX[data_VIX['Date']<last_date]
data_VIX = data_VIX[data_VIX['Date']>first_date]
data_NYT = data_NYT[data_NYT['Date']<last_date]
data_NYT = data_NYT[data_NYT['Date']>first_date]

In [ ]:
data_VIX['Lag 2 Price'] = data_VIX['Price'].shift(-2)
data_VIX['2 Day Change'] = (data_VIX['Lag 2 Price'] - data_VIX['Price'])/data_VIX['Price']

In [ ]:
data_VIX

In [ ]:
data_NYT

In [ ]:
df = data_Obama.groupby('Date').sum()

In [ ]:
dfNYT = dfNYT.groupby('Date').sum()

In [ ]:
df2 = pd.merge(data_VIX, df, how='left', on='Date', suffixes = ('VIX', 'Obama'))
df2 = pd.merge(df2, dfNYT, how='left', on='Date', suffixes = ('', 'NYT'))

In [ ]:
df2

In [ ]:
df2.corr()